In [147]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [2]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 42.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 45.3 MB/s eta 0:00:00:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install nvidia-pyindex
!pip install tritonclient[http]

!pip install -qU pip awscli boto3 sagemaker transformers


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8413 sha256=03d4e5f8b678c8e0714d13efba42d37b3c32e94b7bdefdbcf6d41ea087df0add
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e0/c2/fb/5cf4e1cfaf28007238362cb746fb38fc2dd76348331a748d54
Successfully built nvidia-pyindex
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 341.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 386.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 KB 289.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 353.3 M

### Test Create BERT model from HuggingFace
**If you use from BERT it comes without HEAD o you need to add Head or alternately download from HuggingFace or using Auto so you get with Head**

**Test how to create BERT torchscript model**



In [148]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)
print(f"BERT:Tokenized:Text={tokenized_text}:::")

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
print(f"BERT:indexed_tokens:={indexed_tokens}::")

# -- segments id's
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_model_input = {'input_ids':tokens_tensor, 'attention_mask':segments_tensors }
print(f"BERT:Combining:DICT: all: creating dummy:input:Model:={dummy_model_input}::")

dummy_input = [tokens_tensor, segments_tensors]
print(f"BERT:Finally combining all: creating dummy:input={dummy_input}::")

# Initializing the model with the torchscript flag
# Flag set to True even though it is not necessary as this model does not have an LM Head.
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
    num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, torchscript=True)

# Instantiating the model
model = BertModel(config)

# The model needs to be in evaluation mode for torchscript 
model.eval()

# If you are instantiating the model with `from_pretrained` you can also easily set the TorchScript flag
model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Creating the trace
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])
#torch.jit.save(traced_model, "./bert-uc/traced_bert.pt")

BERT:Tokenized:Text=['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']:::
BERT:indexed_tokens:=[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 103, 2001, 1037, 13997, 11510, 102]::
BERT:Combining:DICT: all: creating dummy:input:Model:={'input_ids': tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])}::
BERT:Finally combining all: creating dummy:input=[tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]]), tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])]::


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
print("Using the BERT Tokensizer::")

print(indexed_tokens)
print(dummy_input)
print(dummy_model_input)


Using the BERT Tokensizer::
[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 103, 2001, 1037, 13997, 11510, 102]
[tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]]), tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])]
{'input_ids': tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])}


dummy_model_input

**Test Tokenizers**

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# load model and tokenizer
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dummy_model_bert_input = tokenizer("This is a sample", return_tensors="pt")

print("Using the BERT:AUTO:TOKENSIZER: Tokenizer::")
print(dummy_model_bert_input) # -- dict -- input id's and attention mask


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Using the BERT:AUTO:TOKENSIZER: Tokenizer::
{'input_ids': tensor([[ 101, 2023, 2003, 1037, 7099,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


### Export as ONYX

In [7]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# export
torch.onnx.export(
    model, 
    tuple(dummy_model_input.values()), #tuple(dummy_model_input.values()),
    f="./bert-uc/torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)


/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/models/distilbert/modeling_distilbert.py:219: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [ ]:
[tokens_tensor, segments_tensors]

In [ ]:
import torch

In [8]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch


# If you are instantiating the model with `from_pretrained` you can also easily set the TorchScript flag
model = BertModel.from_pretrained("bert-base-uncased", torchscript=True)

# Change to eva lmodel
model.eval()

# run a dummy prediction of tokens by tensors
output = model(tokens_tensor)
print(len(output), type(output), type(output[0]))

# Creating the trace
traced_model = torch.jit.trace(model, [tokens_tensor, segments_tensors])
#torch.jit.save(traced_model, "./triton-serve/bert-uc/1/model.pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2 <class 'tuple'> <class 'torch.Tensor'>


In [4]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizing input text
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)
print(f"BERT:Tokenized:Text={tokenized_text}:::")

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
print(f"BERT:indexed_tokens:={indexed_tokens}::")

# -- segments id's -- CAN WE GENERATE THEM via model
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
print(tokens_tensor)
print(segments_tensors)

BERT:Tokenized:Text=['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']:::
BERT:indexed_tokens:=[101, 2040, 2001, 3958, 27227, 1029, 102, 3958, 103, 2001, 1037, 13997, 11510, 102]::
tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])


### Neo Compilers

In [97]:
import sagemaker
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.utils import name_from_base
import boto3
region = boto3.Session().region_name
session = sagemaker.Session()
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")
sess_bucket=session.default_bucket()

In [109]:
[
    torch.randint(1000, (bs, seq_len)).to(device),
    torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]

[tensor([[485, 452, 842, 316, 726, 504, 361, 606, 326, 376,  39, 873, 298, 223,
          265, 844, 180, 805,  17, 714, 446, 305, 274, 512, 797, 704, 966, 181,
          500, 221, 188, 945, 275, 994, 710, 413, 922, 368, 131, 292, 775, 585,
          387, 427,  90, 342, 734, 734, 831, 734, 964, 240, 975, 679, 445, 132,
          161, 403, 379,  54, 585, 384, 359, 947, 861, 951, 855, 527, 576,  67,
          524, 601, 121, 380, 331, 163, 520, 736, 463, 874, 613, 608, 574, 869,
          755, 924, 532, 329, 919, 518, 121,  62,   2, 513, 594, 543,  34, 953,
          314, 460, 980, 702, 443, 176, 934, 260, 458, 364, 825, 859, 309, 726,
          993, 243, 273, 935,  96, 847, 585, 658,  32, 214, 270, 930,   6, 854,
          157, 424, 765, 684, 701, 417, 859, 302, 368,  47, 438, 949, 906, 485,
           82,  92,  95, 482, 504, 976, 335, 160, 933, 610, 349, 980,  57, 964,
          332, 582, 389, 261, 288, 806,   2, 492, 620,  72, 679, 250, 847, 589,
          268, 108,  38, 828, 296, 541, 

In [108]:
dummy_inputs_neo

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [138]:
# Prepare sample input for jit model tracing
seq_0 = "This is just sample text for model tracing, the length of the sequence does not matter because we will pad to the max length that Bert accepts."
seq_1 = seq_0
max_length = 512

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# -- IF you use from bert it comes without HEAD 
model_neo = BertModel.from_pretrained("bert-large-uncased", return_dict=False)
tokenizer_neo = AutoTokenizer.from_pretrained("bert-large-uncased")

tokenized_sequence_pair_neo = tokenizer_neo.encode_plus(
    seq_0, seq_1, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt"
)

example_neo = tokenized_sequence_pair_neo["input_ids"], tokenized_sequence_pair_neo["attention_mask"]

traced_model_neo = torch.jit.trace(model_neo.eval(), example_neo)
#traced_model.save("traced_model/model.pth")

# Just another way of tracing -- or encoding with a single input to the tokenizer
text = "Replace me by any text you'd like."

# Both work
dummy_inputs_neo = tokenizer_neo(text, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
dummy_inputs_neo = tokenizer_neo.encode_plus(text, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")

example_neo = dummy_inputs_neo["input_ids"], dummy_inputs_neo["attention_mask"]

traced_model_neo = torch.jit.trace(model_neo.eval(), example_neo)


torch.jit.save(traced_model_neo, "./inferentia-code/model.pth")

print("Saved {}".format(traced_model_neo))


Using cpu device


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved BertModel(
  original_name=BertModel
  (embeddings): BertEmbeddings(
    original_name=BertEmbeddings
    (word_embeddings): Embedding(original_name=Embedding)
    (position_embeddings): Embedding(original_name=Embedding)
    (token_type_embeddings): Embedding(original_name=Embedding)
    (LayerNorm): LayerNorm(original_name=LayerNorm)
    (dropout): Dropout(original_name=Dropout)
  )
  (encoder): BertEncoder(
    original_name=BertEncoder
    (layer): ModuleList(
      original_name=ModuleList
      (0): BertLayer(
        original_name=BertLayer
        (attention): BertAttention(
          original_name=BertAttention
          (self): BertSelfAttention(
            original_name=BertSelfAttention
            (query): Linear(original_name=Linear)
            (key): Linear(original_name=Linear)
            (value): Linear(original_name=Linear)
            (dropout): Dropout(original_name=Dropout)
          )
          (output): BertSelfOutput(
            original_name=BertSelfO

tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude="*.bin" -zcvf model.tar.gz *

In [139]:
traced_model_neo_url = sagemaker.s3.S3Uploader().upload(
    local_path="./inferentia-code/model.tar.gz",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/neo/bert-uc",
    sagemaker_session=session
)
print(traced_model_neo_url)

s3://sagemaker-us-east-1-622343165275/neo/bert-uc/model.tar.gz


In [140]:
print(traced_model_neo_url)
! aws s3 ls $traced_model_neo_url

s3://sagemaker-us-east-1-622343165275/neo/bert-uc/model.tar.gz
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2022-09-16 18:48:30 1244066527 model.tar.gz


In [ ]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor
from datetime import datetime

prefix = "neuron-experiments/bert-seq-classification"
flavour = "normal"
date_string = datetime.now().strftime("%Y%m-%d%H-%M%S")

compiled_sm_model = PyTorchModel(
    model_data=traced_model_neo_url,
    predictor_cls=Predictor,
    framework_version="1.7", #"1.5.1",
    role=role,
    sagemaker_session=session,
    entry_point="inference_inf1.py",
    #source_dir="inferentia-code",
    py_version="py3",
    name=f"{flavour}-bert-pt181-{date_string}",
    env={"SAGEMAKER_CONTAINER_LOG_LEVEL": "10"},
)


In [144]:
%%time
import json

hardware = "inf1"
flavour = "compiled-inf"
compilation_job_name = f"bert-{flavour}-{hardware}-" + date_string

compiled_inf1_model = compiled_sm_model.compile(
    target_instance_family=f"ml_{hardware}",
    input_shape={"input_ids": [1, 512], "attention_mask": [1, 512]},
    job_name=compilation_job_name,
    role=role,
    framework="pytorch",
    framework_version="1.7", #"1.5.1",
    output_path=f"s3://{sess_bucket}/{prefix}/neo-compilations/{flavour}-model",
    compiler_options=json.dumps("--dtype int64"),
    #     compiler_options={'dtype': 'int64'},    # For compiling to "normal" instance types, cpu or gpu-based
    compile_max_run=900,
)


????????????????????????????.................................................................................................................................................................................*

UnexpectedStatusException: Error for Compilation job bert-compiled-inf-inf1-202209-1618-4850: Failed. Reason: TimeoutException: Compilation stopped due to exceeded time-limit  For further troubleshooting common failures please visit: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-troubleshooting-compilation.html

**End NEO Compilers for Models**

## Start actual BERT with Head for Triton

### This model from HuggingFace does not take the attension ID's so accepts only 1 input
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [512]
  },
  
  
    {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [512]
  }

]


In [9]:
%%writefile triton-serve/bert-uc/config.pbtxt
platform: "pytorch_libtorch"
max_batch_size: 32
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [512]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [512]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [512, 768]
  },
  {
    name: "1634__1"
    data_type: TYPE_FP32
    dims: [768]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 32
}

Overwriting triton-serve/bert-uc/config.pbtxt


### Run for Triton server

**Note**: Amazon SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`. Below is the sample model directory structure

```
bert-uc
├── 1
│   └── model.pt
└── config.pbtxt
```

**Have to use the same Tokenizer to generate the input to test as BERT uncased**

### Create the BERT Model in Torch Script mode -- .pt model
use the ore trained and use torchscript flag here

### Create the LARGE CASE BERT Model in Torch Script using dummy inputs -- .pt model
Create using the dummy inputs

In [10]:
from transformers import GPT2Tokenizer, GPTJModel
from transformers import GPTJForCausalLM, AutoTokenizer

import torch

### Run a simple test for BERT LARGE uncased 

    * We run multiple tests
        * First we token ize and then de tokenize to make sure the vaues match
        * Then we use the model and run predictions to get values
        * Then we run on the traced Model and run predictions to get values 
        * Check to make sure they match

In [47]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', torchscript=True)
model = BertModel.from_pretrained("bert-large-uncased")

text = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs." #"Replace me by any text you'd like."

encoded_input = tokenizer(text, return_tensors='pt')

output = model(**encoded_input)
output

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4472,  0.3378, -0.1825,  ..., -0.8584, -1.3538,  0.4175],
         [-0.2535,  0.1439, -0.1481,  ..., -0.5559, -0.3288, -0.1320],
         [-0.1767, -0.2466, -0.3732,  ..., -0.2728,  0.2910,  0.3882],
         ...,
         [-0.0674, -0.3769,  0.0228,  ...,  0.1540,  0.8440,  0.2331],
         [ 0.0884,  0.4401, -0.8590,  ..., -0.2341, -1.5650, -0.2771],
         [-0.3043,  0.1687, -0.3263,  ..., -0.2537,  0.0570,  0.5401]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9507, -0.9791,  0.9999,  ..., -0.9996,  0.8956, -0.9869]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [40]:
dummy_inputs['input_ids'], dummy_inputs['attention_mask']

(tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [76]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# -- IF you use from bert it comes without HEAD 
model = BertModel.from_pretrained("bert-large-uncased", torchscript=True)
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")


bs = 1
seq_len = 512
dummy_inputs = [
    torch.randint(1000, (bs, seq_len)).to(device),
    torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]
traced_model = torch.jit.trace(model, dummy_inputs)

text = "Replace me by any text you'd like."
dummy_inputs = tokenizer(text, return_tensors='pt', max_length=seq_len, padding=True, truncation=True, )
traced_model = torch.jit.trace(model, dummy_inputs)

model = model.eval()
# model.to(device)
torch.jit.save(traced_model, "./triton-serve/bert-uc/1/model.pt")

print("Saved {}".format(traced_model))

Using cpu device


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2316: UserWarning: `max_length` is ignored w

Saved BertModel(
  original_name=BertModel
  (embeddings): BertEmbeddings(
    original_name=BertEmbeddings
    (word_embeddings): Embedding(original_name=Embedding)
    (position_embeddings): Embedding(original_name=Embedding)
    (token_type_embeddings): Embedding(original_name=Embedding)
    (LayerNorm): LayerNorm(original_name=LayerNorm)
    (dropout): Dropout(original_name=Dropout)
  )
  (encoder): BertEncoder(
    original_name=BertEncoder
    (layer): ModuleList(
      original_name=ModuleList
      (0): BertLayer(
        original_name=BertLayer
        (attention): BertAttention(
          original_name=BertAttention
          (self): BertSelfAttention(
            original_name=BertSelfAttention
            (query): Linear(original_name=Linear)
            (key): Linear(original_name=Linear)
            (value): Linear(original_name=Linear)
            (dropout): Dropout(original_name=Dropout)
          )
          (output): BertSelfOutput(
            original_name=BertSelfO

In [43]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

#### Test encoders various methods

In [24]:
tokenizer(
    "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs.", 
    padding="max_length", 
    max_length=64
   )

{'input_ids': [101, 13012, 2669, 28937, 8241, 3640, 1037, 6112, 1998, 3341, 1999, 7512, 2368, 6129, 5576, 23569, 27605, 5422, 2005, 2119, 17368, 2015, 1998, 14246, 2271, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [25]:
encoded_tokens = tokenizer.encode_plus(
    "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs.",
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = 512,           
    pad_to_max_length = True, # Pad & truncate all sentences
)
#encoded_tokens 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


**Predict test using the traced model Needs Tokens and Attention mask both**

In [26]:
encoded_tokens['input_ids'][0]

101

In [27]:
tokenizer.decode(
    encoded_tokens['input_ids'],
    skip_special_tokens = True, # Add '[CLS]' and '[SEP]'
#     max_length = 512,           
#     pad_to_max_length = True, # Pad & truncate all sentences

)

'triton inference server provides a cloud and edge inferencing solution optimized for both cpus and gpus.'

In [29]:
[encoded_input['input_ids'],encoded_input['attention_mask']]

[tensor([[  101, 13012,  2669, 28937,  8241,  3640,  1037,  6112,  1998,  3341,
           1999,  7512,  2368,  6129,  5576, 23569, 27605,  5422,  2005,  2119,
          17368,  2015,  1998, 14246,  2271,  1012,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0,

In [75]:
import torch
import torch.nn.functional as F

encoded_input = tokenizer(
    "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs.", 
    return_tensors='pt',
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = 100, # -- this model has max length set to 100 -- not to 512,           
    pad_to_max_length = True, # Pad & truncate all sentences

)
#unscripted_output = model.generate( # -- This is for GPTJ version of the model and so uses do_sample, temperature etc 
unscripted_output = model( # -- both work the same way 
    **encoded_input, 
    #inputs=encoded_input['attention_mask']],
    return_dict=True, 
    output_attentions=False, 
    output_hidden_states=False,

    #do_sample=True,
    #temperature=0.9,
    #max_length=128,
)

#tokenizer.decode(unscripted_output[0])
unscripted_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4472,  0.3378, -0.1825,  ..., -0.8584, -1.3538,  0.4175],
         [-0.2535,  0.1439, -0.1481,  ..., -0.5559, -0.3288, -0.1320],
         [-0.1767, -0.2466, -0.3732,  ..., -0.2728,  0.2910,  0.3882],
         ...,
         [ 0.0150,  0.6082,  0.0717,  ...,  0.3446, -0.2089, -0.2529],
         [ 0.0930,  0.2063, -0.1411,  ..., -0.3262, -0.6821, -0.1691],
         [ 0.0200,  0.7166,  0.0530,  ...,  0.1955, -0.1783, -0.2524]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9507, -0.9791,  0.9999,  ..., -0.9996,  0.8956, -0.9869]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [77]:
import torch
import torch.nn.functional as F

encoded_input = tokenizer(
    "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs.", 
    return_tensors='pt',
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = 100, # -- this model has max length set to 100 -- not to 512,           
    pad_to_max_length = True, # Pad & truncate all sentences

)
# Traced Model expects ONLY the INPUT ID's
unscripted_traced_output = traced_model( # -- both work the same way 
    encoded_input['input_ids'] , encoded_input['attention_mask']
)

#tokenizer.decode(unscripted_output[0])
unscripted_traced_output

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(tensor([[[-0.4472,  0.3378, -0.1825,  ..., -0.8584, -1.3538,  0.4175],
          [-0.2535,  0.1439, -0.1481,  ..., -0.5559, -0.3288, -0.1320],
          [-0.1767, -0.2466, -0.3732,  ..., -0.2728,  0.2910,  0.3882],
          ...,
          [ 0.0150,  0.6082,  0.0717,  ...,  0.3446, -0.2089, -0.2529],
          [ 0.0930,  0.2063, -0.1411,  ..., -0.3262, -0.6821, -0.1691],
          [ 0.0200,  0.7166,  0.0530,  ...,  0.1955, -0.1783, -0.2524]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[-0.9507, -0.9791,  0.9999,  ..., -0.9996,  0.8956, -0.9869]],
        grad_fn=<TanhBackward0>))

In [59]:
unscripted_output[1]

tensor([[-0.9045, -0.9739,  0.9999,  ..., -0.9985,  0.8936, -0.9707]],
       grad_fn=<TanhBackward0>)

In [60]:

tokenizer.batch_decode(unscripted_output[1])[0]


'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [20]:
unscripted_output

tensor([[  101, 13012,  2669, 28937,  8241,  3640,  1037,  6112,  1998,  3341,
          1999,  7512,  2368,  6129,  5576, 23569, 27605,  5422,  2005,  2119,
         17368,  2015,  1998, 14246,  2271,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         19101, 26256, 17884,   524, 21002,  7116, 29986, 25690, 24989, 11437,
         25153, 18364, 20228,  8929, 12011, 16143, 15691, 25636, 13067,  9410,
          1842, 11753,  1594, 18910, 12128, 20549,  

### Use the single input to create the traced model in case the model accepts 1 inputs 
### In case Model accepts 2 inputs like BERT then use 2 to create Traced scripting model
**torch.save(model, "./triton-serve/bert-uc/1/model.pt") Does not work**

### UPLOAD of the Model.tar after it has been created correctly by 

Because we share the same model tar with bloom and with bert-uc
rm model.tar.gz in the triton-serve directory


tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude="*.bin" --exclude "*.tar" -zcvf model.tar.gz bert-uc

**Upload the model.tar.gz to S3 location**

In [78]:
import sagemaker
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.utils import name_from_base
import boto3
region = boto3.Session().region_name
session = sagemaker.Session()
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")


In [79]:
s3_model_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path="./triton-serve/model.tar.gz",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-uc",
    sagemaker_session=session
)
s3_mme_model_path='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/'
print(s3_model_path_triton)
print(s3_mme_model_path)

s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-uc/model.tar.gz
s3://sagemaker-us-east-1-622343165275/bloom/triton_models/


#### Start Single Model Triton for starting

**Triton Image download and sagemaker variables**

In [80]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
from sagemaker.utils import name_from_base

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
    account_id=account_id_map[region], region=region, base=base
)
print(triton_image_uri)

785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3


**Model creation**

In [112]:
endpoint_name_p5 = name_from_base(f"p5-bert-uc-")
print(endpoint_name_p5)

container_p5 = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_triton,
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'bert-uc',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response = sm_client.create_model(
    ModelName=endpoint_name_p5, ExecutionRoleArn=role, PrimaryContainer=container_p5
)
print(create_model_response)

p5-bert-uc--2022-09-16-18-09-53-901
{'ModelArn': 'arn:aws:sagemaker:us-east-1:622343165275:model/p5-bert-uc--2022-09-16-18-09-53-901', 'ResponseMetadata': {'RequestId': '1a6f3ab1-2375-4898-86c9-f7c3488a4537', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1a6f3ab1-2375-4898-86c9-f7c3488a4537', 'content-type': 'application/x-amz-json-1.1', 'content-length': '97', 'date': 'Fri, 16 Sep 2022 18:09:54 GMT'}, 'RetryAttempts': 0}}


**Endpoint config**

In [113]:
# Sampling percentage. Choose an integer value between 0 and 100
initial_sampling_percentage = 10                                                                                                                                                                                                                        

# The S3 URI of where to store captured data in S3
s3_capture_upload_path = 's3://sagemaker-us-east-1-622343165275/bloom/triton_models/datacapture'
data_capture_prefix = 'bloom/triton_models/datacapture'

# Specify either Input, Output, or both
capture_modes = ["Input"] # --[ "Input",  "Output" ] 

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_name_p5,
    ProductionVariants=[
        {
            "InstanceType": "ml.g5.8xlarge", #"ml.g4dn.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": endpoint_name_p5,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig= {
        'EnableCapture': True, # Whether data should be captured or not.
        'InitialSamplingPercentage' : initial_sampling_percentage,
        'DestinationS3Uri': s3_capture_upload_path,
        'CaptureOptions': [{"CaptureMode" : capture_mode} for capture_mode in capture_modes] # 
    }
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])


Endpoint Config Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint-config/p5-bert-uc--2022-09-16-18-09-53-901


**Endpoint**

In [114]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name_p5, EndpointConfigName=endpoint_name_p5
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


Endpoint Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint/p5-bert-uc--2022-09-16-18-09-53-901


In [115]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
status = resp["EndpointStatus"]
print("SINGLE:Model:endpoint:Triton:Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
    status = resp["EndpointStatus"]
    print("Single:model:triton:Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Single:model:triton:Status: " + status)

SINGLE:Model:endpoint:Triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: InService
Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint/p5-bert-uc--2022-09-16-18-09-53-901
Single:model:triton:Status: InService


**Now Invoke The endpoint**

In [116]:
import tritonclient.http as httpclient
from transformers import BertTokenizer
import numpy as np


def tokenize_text(text, enc, max_length=512):
    #enc = BertTokenizer.from_pretrained("bert-base-uncased")
    print(f"Tokenize:text:why??::max_length={max_length}::Tokenizer={enc}")
    encoded_text = enc(text, padding="max_length", max_length=max_length)
    return encoded_text["input_ids"], encoded_text["attention_mask"]


def _get_sample_tokenized_text_binary(text, input_names, output_names, enc, max_length=512):
    inputs = []
    outputs = []
    inputs.append(httpclient.InferInput(input_names[0], [1, max_length], "INT32"))
    inputs.append(httpclient.InferInput(input_names[1], [1, max_length], "INT32"))
    indexed_tokens, attention_mask = tokenize_text(text,enc)

    indexed_tokens = np.array(indexed_tokens, dtype=np.int32)
    indexed_tokens = np.expand_dims(indexed_tokens, axis=0)
    inputs[0].set_data_from_numpy(indexed_tokens, binary_data=True)

    attention_mask = np.array(attention_mask, dtype=np.int32)
    attention_mask = np.expand_dims(attention_mask, axis=0)
    inputs[1].set_data_from_numpy(attention_mask, binary_data=True)

    outputs.append(httpclient.InferRequestedOutput(output_names[0], binary_data=True))
    outputs.append(httpclient.InferRequestedOutput(output_names[1], binary_data=True))
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )
    return request_body, header_length


def get_sample_tokenized_text_binary_pt(text, enc, max_length=512):
    return _get_sample_tokenized_text_binary(
        text, ["INPUT__0", "INPUT__1"], ["OUTPUT__0", "1634__1"], enc, max_length
    )


def get_sample_tokenized_text_binary_trt(text, enc):
    return _get_sample_tokenized_text_binary(text, ["token_ids", "attn_mask"], ["output", "1634"], enc, max_length)

def get_decoded_text(tensors_tokens, enc):
    return_text=tokenizer.batch_decode(gen_tokens)[0]
    return return_text

In [149]:
%%time

import json
max_seq_length=512
text_triton = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs."
print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")

input_ids, attention_mask = tokenize_text(text_triton, tokenizer, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="application/octet-stream", Body=json.dumps(payload)
)

output = json.loads(response["Body"].read().decode("utf8"))

print(output.keys())

Leverage the Tokenizer=PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})::max_seq_length=512:: create above when creating the model 
Tokenize:text:why??::max_length=512::Tokenizer=PreTrainedTokenizerFast(name_or_path='bert-large-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
dict_keys(['model_name', 'model_version', 'outputs'])
CPU times: user 211 ms, sys: 27.3 ms, total: 238 ms
Wall time: 519 ms


In [102]:
output['outputs'][0]['data']

[-0.4471989870071411,
 0.337833970785141,
 -0.18249152600765228,
 -0.1867922991514206,
 0.32078203558921814,
 0.8711028099060059,
 0.17235425114631653,
 -0.5893635749816895,
 0.26671886444091797,
 -0.06457437574863434,
 -0.03806910291314125,
 -0.1377534419298172,
 -0.2331186980009079,
 -1.0004020929336548,
 0.2067660540342331,
 0.6279619932174683,
 -0.23859216272830963,
 0.6072220802307129,
 0.08589115738868713,
 -0.47479569911956787,
 -0.016673872247338295,
 0.04392702132463455,
 -0.5811214447021484,
 -0.07257696986198425,
 -0.2536969482898712,
 0.1400335133075714,
 -0.05290088802576065,
 -0.16839829087257385,
 0.6422528028488159,
 -0.21113884449005127,
 -0.2637954354286194,
 0.30578044056892395,
 -0.11204861104488373,
 -0.5515584945678711,
 -0.12893611192703247,
 0.7867868542671204,
 0.05056419223546982,
 0.4449392855167389,
 -0.9341664910316467,
 -0.23110724985599518,
 -0.271898478269577,
 0.34832245111465454,
 0.3622068166732788,
 -0.911349356174469,
 0.2844811975955963,
 0.7075877

In [ ]:
tokenizer.decode(
    torch.tensor(output['outputs'][0]['data']), #tokenizer.decode(unscripted_output[0])
    skip_special_tokens=True,
    clean_up=True
)

### View Captured Data for inference

In [156]:
#Note: It takes a few minutes for the capture data to appear in S3

import boto3

s3_client = boto3.Session().client('s3')

print(f"Data:capture:S_capture_upload_path:={s3_capture_upload_path}:")
# The S3 URI of where to store captured data in S3
print(f"Data:capture:prefix:={data_capture_prefix }")




current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name_p5)
result = s3_client.list_objects(Bucket=sess_bucket, Prefix=current_endpoint_capture_prefix)

capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Data:capture:S_capture_upload_path:=s3://sagemaker-us-east-1-622343165275/bloom/triton_models/datacapture:
Data:capture:prefix:=bloom/triton_models/datacapture
Found Capture Files:
bloom/triton_models/datacapture/p5-bert-uc--2022-09-16-18-09-53-901/AllTraffic/2022/09/16/19/39-48-038-4f7b8ae0-1597-44c0-be0a-3603245c82bc.jsonl
 bloom/triton_models/datacapture/p5-bert-uc--2022-09-16-18-09-53-901/AllTraffic/2022/09/18/17/12-43-677-21d2da2d-80b0-411f-97c4-edb75f0c27ee.jsonl


In [157]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=sess_bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/octet-stream","mode":"INPUT","data":"eyJpbnB1dHMiOiBbeyJuYW1lIjogIklOUFVUX18wIiwgInNoYXBlIjogWzEsIDUxMl0sICJkYXRhdHlwZSI6ICJJTlQzMiIsICJkYXRhIjogWzEwMSwgMjAyMywgMjAwMywgMTAzNywgNTU0MSwgMzAxNSwgNjkxMiwgMTAxMiwgMjkxNywgMTAxMCwgMjAxNywgMTAwNSwgMjIyMiwgMjAyMiwgMjQ0NSwgMTAzNywgMjU3MzIsIDEwMTIsIDIxMTUsIDI0NjYsIDIzMjMsIDIwMjIsIDIyNDEsIDIwMDYsIDE5OTYsIDI1NzMyLCAxMDEyLCAyNTczMiwgMTAyNCwgMTAzNywgMTI0NTksIDI0NjYsIDIwNTUsIDEwMzcsIDExMTcxLCA4MDAwLCAyNDY2LCAxMDI0LCAyMDA2LCAxMDM3LCAyNjAxLCAxOTk4LCAyNDE2NiwgMjMwNSwgMTAxMCwgMTk5NiwgODAwMCwgMTMxNDcsIDE5OTksIDE5OTYsIDYyODEsIDEwMTIsIDEwMiwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCwgMCw

In [ ]:
import json

print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Generate BASELINE Embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
sentence_embeddings = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    with torch.no_grad():
        outputs = model(encoded_dict['input_ids'], encoded_dict['attention_mask'])
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        sentence_embeddings.append(sentence_embedding)

In [ ]:
sentence_embeddings_list = []

for i in sentence_embeddings:
    sentence_embeddings_list.append(i.numpy())

#### Use the Binary Headers for Triton - faster but same results - BUT ERRORS out as NO RESPONSE is recieved 

We can also use binary+json as the payload format to get better performance for the inference call. The specification of this format is provided here.

Note: With the binary+json format, we have to specify the length of the request metadata in the header to allow Triton to correctly parse the binary payload. This is done using a custom Content-Type header application/vnd.sagemaker-triton.binary+json;json-header-size={}.

Please not, this is different from using Inference-Header-Content-Length header on a stand-alone Triton server since custom headers are not allowed in SageMaker.


In [43]:
%%time

import json
max_seq_length=512
text_triton = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs."
print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")

#input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

request_body, header_length = get_sample_tokenized_text_binary_pt(text_triton, enc) # this returns 


response_binary = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, 
    ContentType="application/vnd.sagemaker-triton.binary+json;json-header-size={}".format(header_length), 
    Body=request_body
)

# Parse json header size length from the response
header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
header_length_str = response["ContentType"][len(header_length_prefix) :]

try:
    # Read response body
    result = httpclient.InferenceServerClient.parse_response_body(
        response_binary["Body"].read(), header_length=int(header_length_str)
    )
    output0_data = result.as_numpy("OUTPUT__0")
    output1_data = result.as_numpy("1634__1")
    print(output0_data)
    print(output1_data)
except:
    print("Error in parsing respinse -- probably the body is empty")

Leverage the Tokenizer=PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})::max_seq_length=512:: create above when creating the model 
Tokenize:text:why??::max_length=512::Tokenizer=PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
Error in parsing respinse -- probably the body is empty
CPU times: user 3.46 ms, sys: 9.35 ms, total: 12.8 ms
Wall time: 32.3 ms


In [44]:
endpoint_name_p5

'p5-bert-uc--2022-09-08-03-02-53-774'

In [150]:
%%time
import json

max_seq_length=512
text_triton = """This is a creative writing exercise. Below, you'll be given a prompt. Your story should be based on the prompt.

Prompt: A scary story about a haunted mouse
Story: On a dark and stormy night, the mouse crept in the shadows. """


print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")
input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids}, # -- enc.tokenize(text)}, #
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

max_run = 100
for ii in range(0, max_run):
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name_p5, ContentType="application/octet-stream", Body=json.dumps(payload)
    )

    output_dict = json.loads(response["Body"].read().decode("utf8"))

    # -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
    output_dict.keys()
    time.sleep(0.5)

    #enc.decode(output_dict['outputs'][0]['data'], skip_special_tokens=True)
output_dict.keys()

Leverage the Tokenizer=PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})::max_seq_length=512:: create above when creating the model 
Tokenize:text:why??::max_length=512::Tokenizer=PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
CPU times: user 19 s, sys: 1.82 s, total: 20.9 s
Wall time: 1min 31s


dict_keys(['model_name', 'model_version', 'outputs'])

In [87]:
output_dict['outputs'][0]

{'name': 'OUTPUT__0',
 'datatype': 'FP32',
 'shape': [1, 512, 768],
 'data': [-0.15243282914161682,
  -0.8572331666946411,
  0.06608188152313232,
  -0.20899571478366852,
  0.35779935121536255,
  -0.4324319064617157,
  0.21307486295700073,
  0.7328435778617859,
  0.2850395441055298,
  -0.8913273811340332,
  0.2898162603378296,
  -0.2516830265522003,
  0.17877909541130066,
  0.22467152774333954,
  -0.16646161675453186,
  0.21520552039146423,
  0.4105544686317444,
  0.49988511204719543,
  0.15959863364696503,
  0.11675862967967987,
  0.012811945751309395,
  -0.6604794859886169,
  0.46912506222724915,
  0.11688332259654999,
  0.15712383389472961,
  -0.03815995156764984,
  -0.13969361782073975,
  0.015787124633789062,
  0.11993402242660522,
  0.3402771055698395,
  -0.6780798435211182,
  0.1458730399608612,
  -0.25922656059265137,
  -0.7832548022270203,
  0.28116920590400696,
  -0.12131515890359879,
  -0.28475871682167053,
  -0.2089376002550125,
  -0.11570572853088379,
  0.35242488980293274,

In [ ]:
input_ids
attention_mask 

# open file in write mode
with open(r'./temp-bloom/input_ids.txt', 'w') as fp:
    for item in input_ids:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done input_ids')
    
# open file in write mode
with open(r'./temp-bloom/attention_mask.txt', 'w') as fp:
    for item in attention_mask:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done attention_mask')


### Clean up

In [111]:
sm_client.delete_endpoint(EndpointName=endpoint_name_p5)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name_p5)
sm_client.delete_model(ModelName=endpoint_name_p5)

{'ResponseMetadata': {'RequestId': 'd1348ba3-0c56-4058-b707-54259ae4065d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd1348ba3-0c56-4058-b707-54259ae4065d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 16 Sep 2022 18:07:51 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# general imports
import boto3
import json
import os
import re
import copy
import time
from time import gmtime, strftime
import numpy as np
import datetime
import pprint
import pandas as pd

# sagemaker
import sagemaker
from sagemaker import get_execution_role

# triton
import tritonclient.http as httpclient

# transformers
from transformers import BertTokenizer

# custom CloudWatch
#from cloudwatch import get_endpoint_metrics


In [ ]:
!docker run --gpus=all --rm -it  -v `pwd`/workspace:/workspace nvcr.io/nvidia/pytorch:21.08-py3 /bin/bash generate_models.sh

## START MME for triton 

**Upload first**

### Upload multiple copies for MME

In [ ]:
for ii in range(1,100):
    s3_model_path_triton_mme = sagemaker.s3.S3Uploader().upload(
        local_path="./triton-serve/model.tar.gz",
        desired_s3_uri=f"s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-uc/model-{ii}",
        sagemaker_session=session
    )
s3_model_path_mme='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-uc'
print("MULTIPLE:Uplodas:")
print(s3_model_path_triton_mme)
print(s3_model_path_mme)

In [ ]:
model

In [ ]:
s3_model_path_mme

**Create the model**

In [ ]:
endpoint_name_p5_mme = name_from_base(f"p5-bert-uc-mme")
print(endpoint_name_p5_mme)

container_p5_mme = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_mme,
    'Mode':'MultiModel',
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'model-1',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response_mme = sm_client.create_model(
    ModelName=endpoint_name_p5_mme, ExecutionRoleArn=role, PrimaryContainer=container_p5_mme
)
print(create_model_response_mme)

**Create the Endpoint config**

In [ ]:
create_endpoint_config_response_mme = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_name_p5_mme,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge", #"ml.g4dn.xlarge",ml.g5.8xlarge
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": endpoint_name_p5_mme,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response_mme["EndpointConfigArn"])

**Create the endpoint**

In [ ]:
create_endpoint_response_mme = sm_client.create_endpoint(
    EndpointName=endpoint_name_p5_mme, EndpointConfigName=endpoint_name_p5_mme
)

print("Endpoint Arn: " + create_endpoint_response_mme["EndpointArn"])


In [ ]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5_mme)
status = resp["EndpointStatus"]
print("MME:Model:endpoint:Triton:Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5_mme)
    status = resp["EndpointStatus"]
    print("MME:model:triton:Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("MME:model:triton:Status: " + status)

**Test the end point**

In [ ]:
%%time

import json
max_seq_length=512
text_triton = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs."
print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")
input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5_mme, ContentType="application/octet-stream", Body=json.dumps(payload), TargetModel  = "/model-9/model.tar.gz"
)

output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
output_dict.keys()

enc.decode(output_dict['outputs'][0]['data'], skip_special_tokens=True)

In [ ]:
endpoint_name_p5_mme

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5_mme, ContentType="text/json", Body=json.dumps(payload), TargetModel  = "/model-9/model.tar.gz"
)
output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
output_dict.keys()

**set up in S3 payload to be used for inference load testing**

In [ ]:
max_seq_length=512
text_triton = """
                Create payload JSON and upload it on S3. 
                This will be used by Inference Recommender to run the load test.
              """

input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

print(f"Sample payload to be used with Inference Recommender")
print(payload)

payload_location = "./sample-payload/"
!mkdir -p $payload_location

payload_archive_name = "payload.tar.gz"

with open(payload_location + "request.json", "w") as f:
    json.dump(payload, f)


!cd ./sample-payload/ && tar czvf ../payload.tar.gz *

print(f"payload.tar.gz created at {payload_location}/{payload_archive_name}")

**Upload sample payload to S3**

In [ ]:
s3_sample_data_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path=f"{payload_archive_name}",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_test_data",
    sagemaker_session=session
)
s3_sample_data_path_triton

## Inference Load test set up
### DOES NOT WORK FOR MME -- SO SKIP this section

In [ ]:
ml_domain = "NATURAL_LANGUAGE_PROCESSING"
ml_task = "FILL_MASK"
ml_framework = "PYTORCH"
framework_version = "1.6.0"
model_tested = "bert-base-uncased"

In [ ]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = "pt-triton-benchmark-model-" + ts
model_package_group_name = "pt-triton-benchmark-model-group-" + ts
advanced_job = "pt-triton-benchmark-advanced-job-" + ts

print(f"SageMaker Model Name: {sm_model_name}")
print(f"SageMaker Mode Package Name: {model_package_group_name}")
print(f"SageMaker Advanced Job Name: {advanced_job}")

In [ ]:
s3_model_path_mme

In [ ]:
container_infrec_mme = {
    'Image': triton_image_uri,
    "NearestModelName": model_tested, #'model-1',
    "Framework": ml_framework,
    'ModelDataUrl': s3_model_path_mme,
    #'Mode':'MultiModel',
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'model-1',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    },
}


In [ ]:
model_pacakge_group_response = sm_client.create_model_package_group(
    ModelPackageGroupName=str(model_package_group_name),
    ModelPackageGroupDescription="BERT large uncased Model group for Triton Serving",
)
print(f"Model Registry package group: {model_pacakge_group_response}")


In [ ]:
model_package_version_response = sm_client.create_model_package(
    ModelPackageGroupName=str(model_package_group_name),
    ModelPackageDescription="BERT large uncased Model group for Triton Serving",
    Domain=ml_domain,
    Task=ml_task,
    SamplePayloadUrl=s3_sample_data_path_triton,
    InferenceSpecification={
        "Containers": [container_infrec_mme],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.g4dn.4xlarge",
            "ml.g4dn.4xlarge",
        ],
        "SupportedContentTypes": ["application/octet-stream"],
        "SupportedResponseMIMETypes": ["application/json"],
    },
)
model_package_version_response

In [ ]:
advanced_response = sm_client.create_inference_recommendations_job(
    JobName=advanced_job,
    JobDescription="nlp triton Inference Advanced Recommender Job",
    JobType="Advanced",
    RoleArn=role,
    InputConfig={
        "ModelPackageVersionArn": model_package_version_response["ModelPackageArn"],
        "JobDurationInSeconds": 7200,
        "EndpointConfigurations": [
            #{"InstanceType": "ml.p3.8xlarge"},
            #{"InstanceType": "ml.p3.2xlarge"},
            {"InstanceType": "ml.p2.16xlarge"},
            {"InstanceType": "ml.g4dn.xlarge"},
            {"InstanceType": "ml.g4dn.8xlarge"},
            {"InstanceType": "ml.g4dn.4xlarge"},
            {"InstanceType": "ml.g4dn.2xlarge"},
            {"InstanceType": "ml.g4dn.12xlarge"},
        ],
        "TrafficPattern": {
            "TrafficType": "PHASES",
            "Phases": [
                {
                    "InitialNumberOfUsers": 2,
                    "SpawnRate": 3,
                    "DurationInSeconds": 900,
                },  # simulating 50 users, 2 initial and 3 new users every minute for 16 minutes
            ],  # second phase, we will strt with 50 users, steady traffic for 5 minutes
        },
        "ResourceLimit": {"MaxNumberOfTests": 10, "MaxParallelOfTests": 5},
    },
    StoppingConditions={
        "MaxInvocations": 30000,
        "ModelLatencyThresholds": [{"Percentile": "P95", "ValueInMilliseconds": 500}],
    },
)

print(advanced_response)


In [ ]:
%%time

ended = False
while not ended:
    inference_recommender_job = sm_client.describe_inference_recommendations_job(
        JobName=str(advanced_job)
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        print(f"Inference recommender job status: {inference_recommender_job['Status']} ")
        ended = True
    else:
        print("Inference recommender job in progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Inference recommender job failed ")
    print("Failed Reason: {}".inference_recommender_job["FailedReason"])
else:
    print("Inference recommender job completed")

## Clean up

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name_p5_mme)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name_p5_mme)
sm_client.delete_model(ModelName=endpoint_name_p5_mme)